In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Agentspace Rest API and Python SDK

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/search/agentspace/agentspace_api_and sdk_v1.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fsearch%2Fagentspace%2Fagentspace_api_and%20sdk_v1.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/search/agentspace/agentspace_api_and sdk_v1.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/agentspace/agentspace_api_and sdk_v1.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Dave Wang](https://github.com/wadave/) |

# Overview
This notebook demonstrates how to use the Agentspace REST API and Python SDK to access the search engine.

There are several ways to interact with the Agentspace engine:

- 1. Search Functionality:
 - 1.1 API: search method
 - 1.2 SDK: search method
- 2. Answer Retrieval:
 - 2.1 API: answer method
 - 2.2 SDK: answer method

**Reference:**

- https://cloud.google.com/agentspace/agentspace-enterprise/docs/libraries#client-libraries-usage-python
- https://cloud.google.com/agentspace/agentspace-enterprise/docs/reference/rest

## Get Started

Install the following packages required to execute this notebook.


In [ ]:
%pip3 install --upgrade --quiet google-cloud-discoveryengine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 64.7 MB/s eta 0:00:00


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

# Prerequisuites
- You need to follow [this link](https://cloud.google.com/agentspace/agentspace-enterprise/docs/quickstart-agentspace) to set up datastore and engine first.
- Make a note of your agentspace app engine id (aka, app id)

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run gcloud config list.
* Run gcloud projects list.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [2]:
# TODO use your own project ID
PROJECT_ID = "ace-chatbot-demo"  # @param {type:"string"}

# Set the project id
#! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [3]:
REGION = "us-central1"  # @param {type: "string"}

## Import libraries

In [4]:
import pprint

from google.api_core.client_options import ClientOptions
import google.auth.transport.requests
from google.cloud import discoveryengine_v1 as discoveryengine
import requests

# 1. Search method:

In [5]:
# TODO(developer): Uncomment these variables before running the sample.
project_id = PROJECT_ID
location = "global"  # Values: "global", "us", "eu"
engine_id = "cymbal-bank-agentspace-sea_1738108565573"

## 1.1 Use search RestAPI

In [6]:
# get auth token
creds, _ = google.auth.default()
auth_req = google.auth.transport.requests.Request()
creds.refresh(auth_req)

In [7]:
def get_search_api_response(query):
    """
    Retrieves search results from the Discovery Engine API.

    This function uses the Search endpoint REST API to query the datastore
    and retrieve relevant information based on the provided query.

    Args:
        query: The search query string.

    Returns:
        A requests.Response object containing the API response.  This response
        will typically contain a JSON payload with search results, but may also
        contain error information.  It is the caller's responsibility to
        parse and handle the response appropriately.
    """

    preamble = """You are an assistant to answer question based on datastore resources
       """

    url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/global/collections/default_collection/engines/{engine_id}/servingConfigs/default_search:search"
    resp = requests.post(
        url,
        headers={
            "Content-Type": "application/json",
            "Authorization": "Bearer " + creds.token,
        },
        json={
            "query": query,
            "pageSize": 10,
            "spellCorrectionSpec": {"mode": "AUTO"},
            "contentSearchSpec": {
                "snippetSpec": {"returnSnippet": True},
                "summarySpec": {
                    "summaryResultCount": 3,
                    "includeCitations": True,
                    "ignoreAdversarialQuery": True,
                    "ignoreNonSummarySeekingQuery": True,
                    "ignoreLowRelevantContent": True,
                    "ignoreJailBreakingQuery": True,
                },
            },
        },
    )

    return resp

In [8]:
query = "who founded cymbal bank?"
resp = get_search_api_response(query)

In [9]:
pprint.pprint(resp.json()["summary"]["summaryText"])

('Lewis Cymbal founded Cymbal Bank in 1992. [1] The bank was initially located '
 'in a small office in downtown Los Angeles. [1] Cymbal Bank is a fictional '
 'bank headquartered in the United States. [2] It has branches in Asia, '
 "Europe, Latin America, North America, and Oceania. [2] Cymbal Bank's growth "
 'was rapid, with 10 branches across Los Angeles by 2000. [1] \n')


## 1.2 Use search Python SDK

In [10]:
def get_search_sdk_response(
    project_id: str,
    location: str,
    engine_id: str,
    search_query: str,
    prompt_preamble: str,
) -> list[discoveryengine.SearchResponse]:
    """Retrieves search results from the Discovery Engine using the Python SDK.

    This function queries the specified Discovery Engine with the provided search
    query and configuration options.  It supports specifying a location-specific
    endpoint for the API.

    Args:
        project_id: The Google Cloud project ID.
        location: The location of the Discovery Engine (e.g., "global", "us-central1").
        engine_id: The ID of the Discovery Engine.
        search_query: The search query string.
        prompt_preamble: The preamble to use for search summaries.

    Returns:
        A list of `discoveryengine.SearchResponse` objects containing the search results.
        This list may be empty if no results are found.

    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store

    """

    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)

    # The full resource name of the search app serving config
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"

    # Optional - only supported for unstructured data: Configuration options for search.
    # Refer to the `ContentSearchSpec` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            model_prompt_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelPromptSpec(
                preamble=prompt_preamble
            ),
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="preview",
            ),
        ),
        # -------extractive content-----
        extractive_content_spec=discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
            max_extractive_segment_count=2,
            max_extractive_answer_count=2,
            return_extractive_segment_score=True,
        ),
    )

    # Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    # print(response)

    return response

In [11]:
search_query = "who founded cymbal bank?"
prompt_preamble = " "

In [12]:
response = get_search_sdk_response(
    project_id,
    location,
    engine_id,
    search_query,
    prompt_preamble,
)

In [13]:
response.summary

summary_text: "Lewis Cymbal founded Cymbal Bank in 1992 in Los Angeles. [1]\n"
summary_with_metadata {
  summary: "Lewis Cymbal founded Cymbal Bank in 1992 in Los Angeles.\n"
  citation_metadata {
    citations {
      end_index: 56
      sources {
      }
    }
  }
  references {
    title: "Cymbal Bank Founding Story"
    document: "projects/971101266488/locations/global/collections/default_collection/dataStores/cymbal-bank-datastore_1738108528755/branches/0/documents/481fa3039e1f7d903998c371b71bb06f"
  }
  references {
    title: "Cymbal Bank Organizations & Roles"
    document: "projects/971101266488/locations/global/collections/default_collection/dataStores/cymbal-bank-datastore_1738108528755/branches/0/documents/29d1ba7dab82553580cfd57c491f59f7"
  }
  references {
    title: "Cymbal Bank Business Travel Guide"
    document: "projects/971101266488/locations/global/collections/default_collection/dataStores/cymbal-bank-datastore_1738108528755/branches/0/documents/99a6c8e738ead0ae50b

In [14]:
from google.protobuf.json_format import MessageToDict

document_dict = MessageToDict(
    response.results[0].document._pb, preserving_proto_field_name=True
)

In [15]:
document_dict["derived_struct_data"]["extractive_answers"][0]["content"]

'Cymbal Bank Founding Story Written July 12, 20 Authored by <b>Lewis Cymbal</b>, Founder &amp; CEO Lewis Cymbal founded Cymbal Bank in 1992 in a small office in downtown Los Angeles. The bank quickly grew, by 2000, it had 10 branches across the city.'

# 2. Answer method

## 2.1 Use answer REST API

In [16]:
# refresh token
creds, _ = google.auth.default()
auth_req = google.auth.transport.requests.Request()

creds.refresh(auth_req)

In [17]:
engine_id = "cymbal-bank-agentspace-sea_1738108565573"

In [18]:
def get_answer_api_response(query):
    """Retrieves an answer from the Discovery Engine's Answer endpoint.

    This function sends a POST request to the Discovery Engine's Answer endpoint
    to retrieve an answer based on the provided query.

    Args:
        query: The query string to be answered.

    Returns:
        A `requests.Response` object containing the API response.  The response
        will typically contain a JSON payload with the answer, citations, and
        related questions.  It is the caller's responsibility to parse and
        handle the response appropriately, including checking for errors.
    """

    preamble = """You are an assistant to answer question based on datastore resources
       """

    url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/global/collections/default_collection/engines/{engine_id}/servingConfigs/default_search:answer"
    resp = requests.post(
        url,
        headers={
            "Content-Type": "application/json",
            "Authorization": "Bearer " + creds.token,
        },
        json={
            "query": {"text": query},
            "safetySpec": {
                "enable": False,
            },
            "relatedQuestionsSpec": {
                "enable": True,
            },
            "queryUnderstandingSpec": {
                "queryClassificationSpec": {
                    "types": "ADVERSARIAL_QUERY",
                    "types": "NON_ANSWER_SEEKING_QUERY",
                },
                "queryRephraserSpec": {
                    "disable": False,  # disable the rephraser
                    "maxRephraseSteps": 5,  # controls the max hops we'd like to use for a given query
                },
            },
            "searchSpec": {
                "searchParams": {
                    # Max search result will be put for answer generation.
                    "maxReturnResults": 5,
                    "orderBy": "",
                },
            },
            "answerGenerationSpec": {
                # enable the citations in the output
                "includeCitations": True,
                # force the answer to be generated in a target language.
                "answerLanguageCode": "en",
                "modelSpec": {
                    "modelVersion": "gemini-1.5-flash-001/answer_gen/v1",
                },
                "promptSpec": {"preamble": preamble},
                # "ignoreAdversarialQuery": True,
                # "ignoreNonAnswerSeekingQuery": True,
                # "ignoreLowRelevantContent": True,
            },
        },
    )

    return resp

In [19]:
query = "who founded cymbal bank?"

In [20]:
resp = get_answer_api_response(query)
pprint.pprint(resp.json()["answer"]["answerText"])

('Lewis Cymbal founded Cymbal Bank in 1992 in a small office in downtown Los '
 'Angeles. The bank quickly grew, and by 2000, it had 10 branches across the '
 'city. In 2005, Cymbal Bank was acquired by a larger bank, but Lewis remained '
 'CEO of the combined bank. Lewis was excited about the acquisition, as it '
 'would allow Cymbal Bank to expand its reach and offer its customers more '
 'services. He also saw it as an opportunity to create a more '
 'customer-friendly bank. \n')


## 2.2 Use answer Python SDK

In [21]:
project_id = PROJECT_ID
location = "global"  # Values: "global", "us", "eu"


def get_answer_sdk_response(
    project_id: str,
    location: str,
    engine_id: str,
    query: str,
) -> discoveryengine.AnswerQueryResponse:
    """Retrieves an answer from the Discovery Engine using the AnswerQuery API.

    This function queries the specified Discovery Engine with the provided query
    and configuration options using the `AnswerQuery` API. It supports
    specifying a location-specific endpoint for the API.

    Args:
        project_id: The Google Cloud project ID.
        location: The location of the Discovery Engine (e.g., "global", "us-central1").
        engine_id: The ID of the Discovery Engine.
        query: The query string.

    Returns:
        A `discoveryengine.AnswerQueryResponse` object containing the answer,
        citations, and other related information.

    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    """
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.ConversationalSearchServiceClient(
        client_options=client_options
    )

    # The full resource name of the Search serving config
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_serving_config"

    # Optional: Options for query phase
    # The `query_understanding_spec` below includes all available query phase options.
    # For more details, refer to https://cloud.google.com/generative-ai-app-builder/docs/reference/rest/v1/QueryUnderstandingSpec
    query_understanding_spec = discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec(
        query_rephraser_spec=discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryRephraserSpec(
            disable=False,  # Optional: Disable query rephraser
            max_rephrase_steps=1,  # Optional: Number of rephrase steps
        ),
        # Optional: Classify query types
        query_classification_spec=discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryClassificationSpec(
            types=[
                discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryClassificationSpec.Type.ADVERSARIAL_QUERY,
                discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryClassificationSpec.Type.NON_ANSWER_SEEKING_QUERY,
            ]  # Options: ADVERSARIAL_QUERY, NON_ANSWER_SEEKING_QUERY or both
        ),
    )

    # Optional: Options for answer phase
    # The `answer_generation_spec` below includes all available query phase options.
    # For more details, refer to https://cloud.google.com/generative-ai-app-builder/docs/reference/rest/v1/AnswerGenerationSpec
    answer_generation_spec = discoveryengine.AnswerQueryRequest.AnswerGenerationSpec(
        ignore_adversarial_query=False,  # Optional: Ignore adversarial query
        ignore_non_answer_seeking_query=False,  # Optional: Ignore non-answer seeking query
        ignore_low_relevant_content=False,  # Optional: Return fallback answer when content is not relevant
        model_spec=discoveryengine.AnswerQueryRequest.AnswerGenerationSpec.ModelSpec(
            model_version="gemini-1.5-flash-001/answer_gen/v2",  # Optional: Model to use for answer generation
        ),
        prompt_spec=discoveryengine.AnswerQueryRequest.AnswerGenerationSpec.PromptSpec(
            preamble="Give a detailed answer.",  # Optional: Natural language instructions for customizing the answer.
        ),
        include_citations=True,  # Optional: Include citations in the response
        answer_language_code="en",  # Optional: Language code of the answer
    )

    # Initialize request argument(s)
    request = discoveryengine.AnswerQueryRequest(
        serving_config=serving_config,
        query=discoveryengine.Query(text=query),
        session=None,  # Optional: include previous session ID to continue a conversation
        query_understanding_spec=query_understanding_spec,
        answer_generation_spec=answer_generation_spec,
    )

    # Make the request
    response = client.answer_query(request)

    # Handle the response
    print(response)

    return response

In [22]:
query = "who founded cymbal bank?"
resp = get_answer_sdk_response(
    project_id,
    location,
    engine_id,
    query,
)

answer {
  state: SUCCEEDED
  answer_text: "Lewis Cymbal founded Cymbal Bank in 1992.  He established the bank in a small office in downtown Los Angeles.  By 2000, the bank had expanded to ten branches across the city.  In 2005, Cymbal Bank was acquired by a larger bank, but Lewis Cymbal remained CEO of the combined entity.  Cymbal Bank is described as a fictional bank headquartered in the United States with branches in Asia, Europe, Latin America, North America, and Oceania.\n"
  citations {
    end_index: 41
    sources {
      reference_id: "0"
    }
  }
  citations {
    start_index: 43
    end_index: 109
    sources {
      reference_id: "0"
    }
  }
  citations {
    start_index: 111
    end_index: 174
    sources {
      reference_id: "0"
    }
  }
  citations {
    start_index: 176
    end_index: 281
    sources {
      reference_id: "0"
    }
  }
  citations {
    start_index: 283
    end_index: 436
    sources {
      reference_id: "1"
    }
  }
  references {
    chunk_info

In [23]:
resp.answer.answer_text

'Lewis Cymbal founded Cymbal Bank in 1992.  He established the bank in a small office in downtown Los Angeles.  By 2000, the bank had expanded to ten branches across the city.  In 2005, Cymbal Bank was acquired by a larger bank, but Lewis Cymbal remained CEO of the combined entity.  Cymbal Bank is described as a fictional bank headquartered in the United States with branches in Asia, Europe, Latin America, North America, and Oceania.\n'

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

{TODO: Include commands to delete individual resources below}